In [ ]:
# Library
import pandas as pd
import numpy as np
import calendar
import copy
import datetime
import multiprocessing
import helper as h
from tqdm import tqdm
from itertools import repeat

In [ ]:
df = pd.read_csv('daily_transactions.csv', sep=',')
print(df.shape)
df.head(5)

In [ ]:
df.info()

In [ ]:
# Convert column from obj to datetime
df['event_date'] = pd.to_datetime(df['event_date'])

# Sort dataframe
df = df[df['event_date'] <= '2021-04-30']
df = df.sort_values(by=['_id', 'event_date'])

In [6]:
# unique_id = list(df['_id'].unique())

# with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
#     index = pool.starmap(h.check_usertype, zip(unique_id, repeat(df)))

# # Drop index
# df.drop(index)

In [ ]:
# Store result
months = list(calendar.month_name[1:])
years = [2020, 2021]
newUser = {}

# Create dict to store new user
for y in years:
    for m in months:
        newUser["{}-{}".format(m, y)] = 0

# Copy
dropOff = copy.deepcopy(newUser)
returnUser = copy.deepcopy(newUser)

In [ ]:
# Convert column from obj to datetime
df['event_date'] = pd.to_datetime(df['event_date'])

# Sort dataframe and drop duplicate
df_ft = df.drop_duplicates(subset=['_id'], keep='first')
df_ft = df_ft.sort_values(by='event_date')
df_ft = df_ft[df_ft['event_date'] >= '2020-04-01']

# Add columns
df_ft['first_tran'] = True

# Count new user for each months
for _, row in tqdm(df_ft.iterrows()):
    newUser["{}-{}".format(calendar.month_name[row['event_date'].month], row['event_date'].year)] += 1

# New users
newUser = {k:v for k,v in newUser.items() if v != 0}
newUser

In [ ]:
# Sort dataframe and drop duplicate
df_lt_churn = df.drop_duplicates(subset=['_id'], keep='last')
df_lt_churn = df_lt_churn.sort_values(by='event_date')
df_lt_store = df_lt_churn[df_lt_churn['event_date'] >= '2020-04-01']

# Churn periods
churn = {'A': 360, 'B': 360, 'C': 120, 'D': 260}

# Add the churn values to all of the row
for idx, row in tqdm(df_lt_churn.iterrows()):
    df_lt_churn.at[idx, 'event_date'] = row['event_date'] + datetime.timedelta(days=churn[row['usertype']])

# Filter
df_lt_churn = df_lt_churn[(df_lt_churn['event_date'] >= '2020-04-01') & (df_lt_churn['event_date'] <= '2021-04-30')]

# Count drop off user for each months
for _, row in tqdm(df_lt_churn.iterrows()):
    dropOff["{}-{}".format(calendar.month_name[row['event_date'].month], row['event_date'].year)] += 1

# Old users
dropOff

In [ ]:
# Add columns
df_lt_store['last_tran'] = True

# Filter
df = df[(df['event_date'] >= '2020-04-01') & (df['event_date'] <= '2021-04-30')]

# Merge
df_merge = pd.merge(df_ft, df_lt_store, how="outer", on=['_id', 'event_date', 'usertype'])
df_merge = pd.merge(df_merge, df, how="outer", on=['_id', 'event_date', 'usertype'])
df_merge = df_merge.replace(np.nan, False)

# Remove value with single transaction (true & true)
df_merge = df_merge[(df_merge['first_tran'] != True) | (df_merge['last_tran'] != True)]
df_merge = df_merge[(df_merge['first_tran'] != True)]
df_merge

In [ ]:
# Sort dataframe and drop duplicate
df_ft_churn = df.drop_duplicates(subset=['_id'], keep='first')
df_ft_churn = df_ft_churn.sort_values(by='event_date')

# Filter
df_ft_churn = df_ft_churn[df_ft_churn['event_date'] <= '2021-04-30']
df_ft_churn.sort_values(by='event_date')

In [ ]:
# Convert to numpy array
user_ids = list(df_merge['_id'].unique())
np_merge = df_merge.to_numpy()
np_ft_churn = df_ft_churn.to_numpy()

In [ ]:
# Run multiprocessing
with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
    result = pool.starmap(h.find_return_user, zip(user_ids, repeat(np_merge), repeat(np_ft_churn), repeat(dropOff), repeat(returnUser)))    

# Print result
returnUser = {k:v for k,v in result[1].items() if v != 0}
returnUser

In [ ]:
# Print result
dropOff = {k:v for k,v in result[0].items() if v != 0}
dropOff

In [ ]:
# test = {'_id': ['abc', 'abc', 'abc', 'abc'], 
#         'event_date': ['2020-04-01', '2020-10-20', '2020-10-21', '2021-04-29'], 
#         'usertype': ['C', 'C', 'C', 'C'], 
#         'first_tran': [True, False, False, False], 
#         'last_tran': [False, False, False, True]
#     }

# # Churn periods
# churn = {'A': 360, 'B': 360, 'C': 120, 'D': 260}

# df_test = pd.DataFrame(test)
# df_test['event_date'] = pd.to_datetime(df_test['event_date'])

# churn_period = datetime.time()
# for idx, row in df_test.iterrows():
#     if row['first_tran'] == True:
#         churn_period = row['event_date'] + datetime.timedelta(days=churn[row['usertype']])
#         continue
    
#     if row['event_date'] < churn_period:
#         continue
#     else:
#         dropOff["{}-{}".format(calendar.month_name[churn_period.month], churn_period.year)] += 1
#         returnUser["{}-{}".format(calendar.month_name[row['event_date'].month], row['event_date'].year)] += 1
#         churn_period = row['event_date'] + datetime.timedelta(days=churn[row['usertype']])

# returnUser = {k:v for k,v in returnUser.items() if v != 0}
# dropOff = {k:v for k,v in dropOff.items() if v != 0}

# print(dropOff, returnUser)

In [ ]:
# # Grouping
# df_a = df_sep.loc[df['usertype'] == 'A']
# df_b = df_sep.loc[df['usertype'] == 'B']
# df_c = df_sep.loc[df['usertype'] == 'C']
# df_d = df_sep.loc[df['usertype'] == 'D']

# # Data Summary
# display = pd.DataFrame({'usertype': ['A', 'B', 'C', 'D'], 'unique': [len(df_a['_id'].unique()), len(df_b['_id'].unique()), len(df_c['_id'].unique()), len(df_d['_id'].unique())], 'count': [len(df_a), len(df_b), len(df_c), len(df_d)]})
# display